In [2]:
import akshare as ak
gdp = ak.macro_china_gdp()
addData = gdp[gdp["季度"]>="2021-06-01"]
print(addData[gdp])

ValueError: Boolean array expected for the condition, not object

In [ ]:
run erfpipe.py
